<a href="https://colab.research.google.com/github/Ahlawatash/Weather-Assignment/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [5]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install requests
!pip install matplotlib
!pip install pyinputplus
!pip install pandas
!pip install hands-on-ai


In [15]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = '29e060f713934fef81330206ee7b9842.m8axTsN_eQbon2ajZvmLV42e'

## 📦 Setup and Configuration
Import required packages and setup environment.

In [16]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
import pandas as pd
from datetime import datetime


# Add any other setup code here

## 🌤️ Weather Data Functions

In [18]:
# Define get_weather_data() function here
# 🌦️ Weather Data Functions + Interactive Menu

import requests

WTTR_URL = "https://wttr.in/{loc}?format=j1"

def _safe_float(x, default=None):
    """Safely convert a value to float (or return None if missing)."""
    try:
        return float(x)
    except Exception:
        return default


def get_current_weather(location: str) -> dict:
    """
    Return current weather for a location using wttr.in (no API key).
    Keys returned: time, temperature_c, feels_like_c, weather_text, humidity_pct, precip_mm, wind_kmh
    """
    url = WTTR_URL.format(loc=location)
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    data = r.json()

    cc = (data.get("current_condition") or [{}])[0]

    desc_list = cc.get("weatherDesc") or []
    weather_text = desc_list[0].get("value") if isinstance(desc_list, list) and desc_list else None

    return {
        "location": location,
        "time": cc.get("localObsDateTime") or cc.get("observation_time"),
        "temperature_c": _safe_float(cc.get("temp_C")),
        "feels_like_c": _safe_float(cc.get("FeelsLikeC")),
        "weather_text": weather_text,
        "humidity_pct": _safe_float(cc.get("humidity")),
        "precip_mm": _safe_float(cc.get("precipMM")),
        "wind_kmh": _safe_float(cc.get("windspeedKmph")),
    }


def get_weather_data(location, forecast_days=3):
    """
    Retrieve current + multi-day forecast for a specified location (1–5 days).
    """
    url = WTTR_URL.format(loc=location)
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    data = r.json()

    current = get_current_weather(location)

    days = data.get("weather") or []
    forecast = []
    for d in days[:forecast_days]:
        forecast.append({
            "date": d.get("date"),
            "min_temp": _safe_float(d.get("mintempC")),
            "max_temp": _safe_float(d.get("maxtempC")),
            "avg_humidity": sum(_safe_float(h.get("humidity"), 0) for h in d["hourly"]) / len(d["hourly"]),
            "precip_mm": sum(_safe_float(h.get("precipMM"), 0) for h in d["hourly"]),
        })

    return {"current": current, "forecast": forecast}


# ------------------ Interactive section ------------------

def _print_card(now: dict):
    """Pretty print full weather card."""
    print(
        f"📍 {now['location']}\n"
        f"🕒 {now['time']}\n"
        f"🌡️ Temp: {now['temperature_c']} °C (feels {now['feels_like_c']} °C)\n"
        f"⛅ {now['weather_text']}\n"
        f"💧 Humidity: {now['humidity_pct']} %   🌧️ Precip: {now['precip_mm']} mm\n"
        f"🌬️ Wind: {now['wind_kmh']} km/h"
    )


def _print_attribute(now: dict, choice: str):
    """Print a single attribute based on menu choice."""
    choice = choice.lower()
    if choice == "temperature":
        print(f"🌡️ Temperature: {now['temperature_c']} °C (feels {now['feels_like_c']} °C)")
    elif choice == "precipitation":
        print(f"🌧️ Precipitation: {now['precip_mm']} mm")
    elif choice == "weather":
        print(f"⛅ Weather: {now['weather_text']}")
    elif choice == "humidity":
        print(f"💧 Humidity: {now['humidity_pct']} %")
    elif choice == "all":
        _print_card(now)
    else:
        print("❓ Unknown choice")


def run_current_weather_cli(default_city: str = None):
    """
    Interactive program:
      - Ask user for a city
      - Show menu: Temperature / Precipitation / Weather / Humidity / All / Change city / Quit
      - Fetches live data from wttr.in
    """
    city = default_city

    while True:
        if not city:
            city = input("\nEnter a city (e.g., Sydney, Delhi, New York): ").strip()
            if not city:
                print("Please enter a valid city name.")
                continue

        try:
            now = get_current_weather(city)
        except Exception as e:
            print(f"⚠️ Could not fetch data for '{city}'. Error: {e}")
            city = None
            continue

        print("\nChoose an option:")
        print(" 1) Temperature\n 2) Precipitation\n 3) Weather\n 4) Humidity\n 5) All\n 6) Change city\n 7) Quit")
        choice = input("Enter 1–7: ").strip()

        if choice == "1":
            _print_attribute(now, "temperature")
        elif choice == "2":
            _print_attribute(now, "precipitation")
        elif choice == "3":
            _print_attribute(now, "weather")
        elif choice == "4":
            _print_attribute(now, "humidity")
        elif choice == "5":
            _print_attribute(now, "all")
        elif choice == "6":
            city = input("Enter a new city: ").strip()
        elif choice == "7":
            print("👋 Bye!")
            break
        else:
            print("❌ Invalid option, please try again.")


# 👉 Run this to start the interactive program
run_current_weather_cli()


print(
    f"📍 {now['location']}\n"
    f"🕒 {now['time']}\n"
    f"🌡️ Temp: {now['temperature_c']} °C (feels {now['feels_like_c']} °C)\n"
    f"⛅ {now['weather_text']}\n"
    f"💧 Humidity: {now['humidity_pct']} %   🌧️ Precip: {now['precip_mm']} mm\n"
    f"🌬️ Wind: {now['wind_kmh']} km/h"
)



📍 Perth
🕒 2025-10-17 12:38 PM
🌡️ Temp: 31.0 °C (feels 33.0 °C)
⛅ Sunny
💧 Humidity: 24.0 %   🌧️ Precip: 0.0 mm
🌬️ Wind: 17.0 km/h


## 📊 Visualisation Functions

In [20]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
from datetime import datetime
import matplotlib.pyplot as plt

def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create a line chart of daily min/max temperatures.

    Accepts either:
      - weather_data["forecast"] with keys: date, min_temp, max_temp
      - weather_data["daily"]    with keys: date, t_min_c, t_max_c

    Args:
        weather_data (dict): The processed weather data.
        output_type (str): 'display' to show the figure, or 'figure' to return it.

    Returns:
        matplotlib.figure.Figure if output_type == 'figure', else None.
    """
    # 1) Extract a normalized list of days
    days = weather_data.get("forecast")
    using_forecast = True
    if not days:
        days = weather_data.get("daily", [])
        using_forecast = False

    if not days:
        raise ValueError("No daily/forecast data available to plot.")

    # 2) Build X (dates) and Y (min/max) series
    X = []
    Ymin = []
    Ymax = []
    for d in days:
        date_str = d.get("date")
        if not date_str:
            continue
        # wttr dates are YYYY-MM-DD
        try:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
        except Exception:
            # Fallback for ISO strings
            dt = datetime.fromisoformat(date_str)
        X.append(dt)

        if using_forecast:
            Ymin.append(d.get("min_temp"))
            Ymax.append(d.get("max_temp"))
        else:
            Ymin.append(d.get("t_min_c"))
            Ymax.append(d.get("t_max_c"))

    if not X:
        raise ValueError("No valid dates to plot.")

    # 3) Title location (best-effort)
    loc = (
        weather_data.get("current", {}).get("location")
        or weather_data.get("location", {}).get("name")
        or "Selected location"
    )

    # 4) Plot (1 figure, no custom colors)
    fig = plt.figure(figsize=(9, 5))
    plt.plot(X, Ymax, marker='o', label="Max temp (°C)")
    plt.plot(X, Ymin, marker='o', label="Min temp (°C)")
    plt.title(f"Temperatures — {loc}")
    plt.xlabel("Date")
    plt.ylabel("°C")
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=30)

    if output_type == 'figure':
        return fig
    plt.show()


def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create a bar chart of daily total precipitation (mm).

    Accepts either:
      - weather_data["forecast"] with key: precip_mm
      - weather_data["daily"]    with key: precip_mm

    Args:
        weather_data (dict): The processed weather data.
        output_type (str): 'display' to show the figure, or 'figure' to return it.

    Returns:
        matplotlib.figure.Figure if output_type == 'figure', else None.
    """
    # 1) Extract days from either forecast or daily
    days = weather_data.get("forecast") or weather_data.get("daily", [])
    if not days:
        raise ValueError("No daily/forecast data available to plot.")

    # 2) Build X (dates) and Y (precip) series
    X = []
    P = []
    for d in days:
        date_str = d.get("date")
        if not date_str:
            continue
        try:
            dt = datetime.strptime(date_str, "%Y-%m-%d")
        except Exception:
            dt = datetime.fromisoformat(date_str)
        X.append(dt)
        P.append(d.get("precip_mm"))

    if not X:
        raise ValueError("No valid dates to plot.")

    # 3) Title location (best-effort)
    loc = (
        weather_data.get("current", {}).get("location")
        or weather_data.get("location", {}).get("name")
        or "Selected location"
    )

    # 4) Plot (1 figure, no custom colors)
    fig = plt.figure(figsize=(9, 5))
    plt.bar(X, P)
    plt.title(f"Precipitation — {loc}")
    plt.xlabel("Date")
    plt.ylabel("Total precip (mm)")
    plt.grid(True, axis='y')
    plt.xticks(rotation=30)

    if output_type == 'figure':
        return fig
    plt.show()



In [10]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

## 🤖 Natural Language Processing

In [11]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [12]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [13]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [14]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.